In [14]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import layers
import numpy as np

In [3]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']
targets = [[1], [0], [1], [1], [0], [1]] 

In [4]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)

In [5]:
print(tokenizer.word_index)

{'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}


In [6]:
sequences = tokenizer.texts_to_sequences(samples)
print(sequences)

[[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]


In [7]:
input_sequences = np.array(sequences)
labels = np.array(targets)

In [9]:
word_index = tokenizer.word_index

In [10]:
print(input_sequences)

[[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]


In [11]:
print(labels)

[[1]
 [0]
 [1]
 [1]
 [0]
 [1]]


In [12]:
batch_size = 2
num_epochs = 100

In [13]:
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

In [15]:
model = tf.keras.Sequential() #모듈 임포트 안하고 이렇게 써도됨!
model.add(layers.Embedding(vocab_size, emb_size, input_length=4)) #vocab_size를 emb_size차원으로 임베딩
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), #함수로 써도되고, 클래스로 써도됨.
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            2688      
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [17]:
# (None, 4, 128) 
# reduce_mean : 특정축으로 차원을 줄여서 평균을 구해줌
# axis=1이 4이므로 4가 없어짐 (None, 128) 이렇게
# flatten은 데이터 손실 없이 차원을 줄이는거

In [18]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 0s/step - loss: 0.6941 - accuracy: 0.3333 
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6755 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6600 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6264 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6062 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 998us/step - loss: 0.5818 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5550 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5215 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4809 - accuracy: 1.0000
Epoch 11/100
3/3 [=========

In [19]:
class CustomModel(tf.keras.Model):
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [20]:
model = CustomModel(vocab_size=vocab_size, embed_dimension=emb_size, 
                    hidden_dimension=hidden_dimension, output_dimension=output_dimension)

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), #함수로 써도되고, 클래스로 써도됨.
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 0s/step - loss: 0.6951 - accuracy: 0.1667 
Epoch 2/100
3/3 [==============================] - 0s 6ms/step - loss: 0.6774 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6607 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6283 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 997us/step - loss: 0.6057 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5831 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5552 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5246 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4864 - accuracy: 1.0000
Epoch 11/100
3/3 [=========